# Makine Öğrenimi Projesi Bölüm - 3
Bu projede gerçek veri setiyle uğraşan eksiksiz bir makine öğrenim problemi üzerinde çalışıyoruz. Bina enerji verilerini kullanarak, bir binanın "Enerji Yıldızı Puanını" tahmin etmek için bir model oluşturmaya çalışıyoruz, bu işlemi de denetimli bir gerileme(supervised regression), makine öğrenimi görevi tarafından gerçekleştiriyoruz.

Bu projenin ilk iki bölümünde, makine öğrenme işlemlerinin ilk 6 adımını uyguladık:

1.Veri temizleme ve biçimlendirme
2.Keşifsel veri analizi
3.Özellik mühendisliği ve seçimi
4.Bir performans metriğinde çeşitli makine öğrenim modellerinin karşılaştırılması
5.Problem için optimize etmek üzere en iyi modele hiperparametre ayarlaması yapılması
6.Test setindeki en iyi modelin değerlendirilmesi
7.Model sonuçlarının en iyi şekilde yorumlanması
8.Sonuçların grafiğini oluşturun ve iyi bir rapor yazın

Bu not defterinde, son iki adıma yoğunlaşacağız ve kurduğumuz modelin kara kutusuna bakmaya çalışacağız. Oluşturduğumuz modelin doğru olduğunu biliyoruz, çünkü Energy Yıldız Puanının gerçek değeri 9.1 noktası içinde öngörebiliyor, ancak bu tahminleri tam olarak nasıl yapıyor? Gradient destek makinesini(Gradient Boosting Machine) anlamaya çalışmanın bazı yollarını inceleyeceğiz ve daha sonra sonuçlar çıkartacağız.

Kütüphaneleri eklemekle kodlama çalışmamıza başlıyoruz, Bu çalışmada standart veri bilimi kütüphanelerini kullanacağız.

In [25]:
# Verileri işlemek için Numpy ve Pandas Kütüphanelerini yüklüyoruz
import lime
import lime.lime_tabular
import pandas as pd
import numpy as np

# Bölümler arasında uyarı olmaması için uyarıları kapatıyoruz
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 60)

# Görselleştirme için matplotlib kütüphanesini dahil ediyoruz
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(font_scale = 2)
# Yazı boyutlarını varsayılan  olarak 24 punto ayarlıyoruz
plt.rcParams['font.size'] = 24

from IPython.core.pylabtools import figsize

# Eksik değerlere etki edecek kütüphaneyi yüklüyoruz
from sklearn.preprocessing import Imputer, MinMaxScaler

# Makine Öğrenmesi modellerini dahil ediyoruz
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import tree

#Tahminleri açıklamak için  LIME kütüphanesini ekliyoruz
import lime
import lime.lime_tabular

## Eğitim ve Test Verilerini alıyoruz

In [ ]:
# Veriçerçevesinden verileri okuyoruz
train_features = pd.read_csv('../input/nyctrain-test-data/training_features.csv')
test_features = pd.read_csv('../input/nyctrain-test-data/testing_features.csv')
train_labels = pd.read_csv('../input/nyctrain-test-data/training_labels.csv')
test_labels = pd.read_csv('../input/nyctrain-test-data/testing_labels.csv')

## Modeli Tekrar oluşturuyoruz

In [ ]:
#Median doldurma stratejisi ile bir imputer nesne oluşturuyoruz
imputer = Imputer(strategy = 'median')

#Eğitim verilerine imputer nesnesini uyguluyoruz
imputer.fit(train_features)

#Eğtim ve Test verilerini dönüştürüyoruz
X = imputer.transform(train_features)
X_test = imputer.transform(test_features)

#Tek boyutlu vektör için Sklearn de etiketleme yapıyoruz
y = np.array(train_labels).reshape((-1,))
y_test = np.array(test_labels).reshape((-1,))

In [ ]:
#Ortalama mutlak hata'yı(MAE) bulmak için fonksiyonumuz
def mae(y_true, y_pred):
    return np.mean(abs(y_true - y_pred))

In [ ]:
model = GradientBoostingRegressor(loss = 'lad', max_depth = 5, max_features = None,
                                 min_samples_leaf = 6, min_samples_split = 6,
                                 n_estimators = 800, random_state = 42)
model.fit(X, y)

In [ ]:
#Test verileri üzerinde tahminlerimizi yapıyoruz
model_pred = model.predict(X_test)

print('Test verileri üzerinde son modelimizin performans testi: MAE = %0.4f' % mae(y_test, model_pred))

## Modeli yorumlamak
Makine öğrenimi çoğu zaman bir kara kutu olarak eleştirilmektedir: Bir tarafa veri koyduk ve bize diğer tarafta cevapları verdi. Bu cevaplar genellikle son derece doğru olsa da, model bize tahminleri gerçekten nasıl yaptığını anlatmıyor. Bu bir dereceye kadar doğrudur, ancak bir modelin Yerel Olarak Yorumlanabilir Model-agnostik Açıklayıcı (LIME) gibi nasıl düşündüğünü keşfedip deneyebileceğimiz yollar vardır. Bu, tahmin edilebilir bir model olan, tahmin etrafında doğrusal bir regresyon öğrenerek model tahminlerini açıklamayı amaçlamaktadır!

Modelimizi yorumlamak için birkaç yolu keşfedeceğiz:

-  Özelliklerin önemi
- Lokal olarak Yorumlanabilir Model-agnostik Explainer (LIME)
- Topluluktaki tek bir karar ağacını incelemek.

### Özellik Önemi
Karar ağaçlarının bir grubunu yorumlayabilmemizin temel yollarından biri, özellik itirazları olarak bilinen şeydir. Bunlar, hedefin en çok tahmin edilen değişkenleri olarak yorumlanabilir. Öznitelik önemlerinin asıl ayrıntıları oldukça karmaşık olsa da burada konuyla ilgili bir soru var, özellikleri karşılaştırmak ve problemimizle en uygun olanı belirlemek için göreceli değerleri kullanabiliriz.

Özelliklerin eğitimli bir topluluktan alınması, scikit-öğrenmede oldukça kolaydır. Bunları analiz etmek ve görselleştirmek için özellikteki bir veri çerçevesine saklayacağız.

In [ ]:
#Veri çerçevesi içerisinden önemli özellikleri ayırıyoruz
features_results = pd.DataFrame({'features': list(train_features.columns),
                                'importance': model.feature_importances_})
#içlerinden en önemli 10 tanesine bakalım
features_results = features_results.sort_values('importance', ascending = False).reset_index(drop = True)

features_results.head(10)

Enerji Kullanımı Yoğunluğu (Site EUI (kBtu / ft²)) ve Hava Durumu Normalize Saha Elektrik Yoğunluğu, Hava Normalleştirilmiş Yer Elektrik Yoğunluğu (kWh / ft²), oldukça büyük bir marjla en önemli iki özelliktir. Bundan sonra, göreceli önem, önemli ölçüde düşer ve bu da neredeyse aynı performansa sahip bir model oluşturmak için tüm özelliklerin korunmasına ihtiyaç duymayacağımızı gösterir.

Görsel olarak karşılaştırmak için özellik önemlerini görselleştirelim.

In [ ]:
figsize(12, 10)
plt.style.use('fivethirtyeight')

#En önemli 20 özelliği yatay bar çubuğundan görselleştirelim
features_results.loc[:9, :].plot(x = 'features', y = 'importance',
                               edgecolor = 'k',
                               kind = 'barh', color = 'yellow');
plt.xlabel('Bağıl Önem', size = 20); 
plt.ylabel('')
plt.title('Rastgele Ormanda Özelliklerin önemi', size = 30);

## Özellik Seçimi için Önemli Özellikleri Kullanma
Her özelliğin skoru bulmak için önemli olmadığı göz önüne alındığında, rasgele ormandaki en önemli özelliklerin alt kümesiyle doğrusal bir regresyon gibi daha basit bir model kullansaydık ne olurdu? Doğrusal regresyon, taban çizgisini geride bıraktı, ancak model karmaşık modellere kıyasla iyi performans göstermedi. Performansın iyileştirilip iyileştirilmediğini görmek için doğrusal regresyonda sadece en önemli 10 özelliği kullanmayı deneyelim. Ayrıca bu özellikleri sınırlayabilir ve rastgele ormanı yeniden değerlendirebiliriz.

In [ ]:
#En Önemli Özellikleri ayıklıyoruz
most_important_features = features_results['features'][:10]

#Herbir özellik isminin indexini bulalım
indices = [list(train_features.columns).index(x) for x in most_important_features]

#Sadece en önemli özellikler kalıyor
X_reduced =X[:, indices]
X_test_reduced = X_test[:, indices]

print('Eğitim setindeki en önemli özelliklerin şekli: ', X_reduced.shape)
print('Test setindeki en önemli özelliklerin şekli: ', X_test_reduced.shape)

In [ ]:
lr = LinearRegression()

#Özelliklerin tamamına uyguluyoruz
lr.fit(X, y)
lr_full_pred = lr.predict(X_test)

#Azaltılmış özelliklere uyguluyoruz
lr.fit(X_reduced, y)
lr_reduced_pred = lr.predict(X_test_reduced)

#Sonuçları görüntülüyoruz
print('Doğrusal Regresyonun tüm sonuçları: MAE =  %0.4f.' % mae(y_test, lr_full_pred))
print('Doğrusal Regresyonun azaltılmış sonuçları : MAE = %0.4f.' % mae(y_test, lr_reduced_pred))

Eh, özelliklerin azaltılması lineer regresyon sonuçlarını geliştirmedi! Düşük önem taşıyan özelliklerde ekstra bilgilerin gerçekte performansı artırdığı ortaya çıkmaktadır.

Gradient(Eğim) artırılmış regresördeki azaltılmış özellik setini kullanalım. Bakalım modelimizin performans nasıl etkilenir?

In [ ]:
#Aynı hiperparametreler'den oluşan bir model oluşturuyoruz
model_reduced = GradientBoostingRegressor(loss = 'lad', max_depth = 5, max_features = None,
                                       min_samples_leaf = 6, min_samples_split =6, 
                                       n_estimators = 800, random_state = 42)
#Özellikleri uygulayıp test ediyoruz
model_reduced.fit(X_reduced, y)
model_reduced_pred = model_reduced.predict(X_test_reduced)

print('Eğimi artırılmışların azaltılmış sonuçları: MAE = %0.4f' % mae(y_test, model_reduced_pred))

Model sonuçları, azaltılmış özellik seti ile biraz daha kötüdür ve son model için tüm özellikleri koruyacağız. Özelliklerin sayısını azaltma arzusu, her zaman en karmaşık modeli oluşturmak istediğimiz içindir: yani, yeterli performansa sahip en basit model. Daha az özellik kullanan bir model daha hızlı eğitilecek ve genellikle daha kolay yorumlanacaktır. Bu durumda, tüm özelliklerin korunması önemli bir sorun değildir çünkü eğitim süresi önemli değildir ve hala birçok özellik ile yorum yapabiliriz.

## Lokal olarak Yorumlanabilir Model-agnostik Açıklamalar
Model tarafından yapılan bireysel tahminleri açıklamak için LIME kullanımına bakacağız. LIME, bir makine öğrenim modelinin bölgeyi yaklaşık bir doğrusal modelle bir tahmin etrafında tahmin ederek nasıl düşündüğünü göstermeye yönelik nispeten yeni bir yaklaşımdır.

Örnekle ilgili tahminleri açıklamaya çalışacağız, model çok yanlış ve örnek doğru bir örnek olabilir. Yorumlanabilirliğe yardımcı olmak için azaltılmış 10 özellik setini kullanmaya kendimizi kısıtlayacağız. En önemli 10 özellik üzerinde eğitilen model biraz daha az doğrudur, ancak genellikle yorumlanabilirlik için doğrulukla işimizi yapmak zorundayız!

In [ ]:
#Azaltılmış değerleri buluyoruz
residuals = abs(model_reduced_pred - y_test)

#En iyi ve en kötü sonuçları görüntülüyoruz
wrong = X_test_reduced[np.argmax(residuals), :]
right = X_test_reduced[np.argmin(residuals), :]

In [ ]:
# Açıklayıcı(explainer) nesnesi oluşturuyoruz
explainer = lime.lime_tabular.LimeTabularExplainer(training_data = X_reduced, 
                                                   mode = 'regression',
                                                   training_labels = y,
                                                   feature_names = list(most_important_features))

In [ ]:
# Display the predicted and true value for the wrong instance
print('Prediction: %0.4f' % model_reduced.predict(wrong.reshape(1, -1)))
print('Actual Value: %0.4f' % y_test[np.argmax(residuals)])

# Explanation for wrong prediction
wrong_exp = explainer.explain_instance(data_row = wrong, 
                                       predict_fn = model_reduced.predict)

# Plot the prediction explaination
wrong_exp.as_pyplot_figure();
plt.title('Explanation of Prediction', size = 28);
plt.xlabel('Effect on Prediction', size = 22);

In [ ]:
wrong_exp.show_in_notebook(show_predicted_value=False)

Bu örnekte, gradyan destekli modelimiz 12.86'lık bir skor öngördü ve gerçek değer 100'dü.

LIME'den gelen görsel  bize örnek için her bir özellikten nihai tahminin katkısını ne olduğunu gösteriyor. Site EUI'sının tahminleri 95.50'nin üzerinde olması nedeniyle önemli ölçüde  tahminleri azalttığını görebiliriz. Hava Normalleştirilmiş Saha Elektrik Şiddeti, diğer taraftan, 3.80'den daha düşük olduğu için tahmini arttırdı.

Bunu, modelimizin Enerji Yıldızı Puanının, Site EUI'sının yüksek olması nedeniyle olduğundan daha düşük olacağını düşündüğünü söyleyebiliriz. Ancak, bu durumda, EUI'nın yüksek değerine rağmen skor 100'dü. Bu kayda değer bir hata. Başlangıçta kafa karıştırıcı olabilirken, şimdi gerçekte, modelin problemle akıl yürüttüğünü ve yanlış değere ulaştığını görebiliriz! Aynı süreçten geçen bir insan muhtemelen aynı sonuca varırdı.(eğer tüm verilerden geçmek için sabır gösterebildiyse).

Şimdi aynı süreci, modelin doğru olduğunu tahmin ederek inceleyebiliriz.

In [ ]:
# Tahmin edilen ve gerçek değerlerin nasıl göründüğüne bakalım
print('Tahmin: %0.4f' % model_reduced.predict(right.reshape(1, -1)))
print('Gerçek Değer: %0.4f' % y_test[np.argmin(residuals)])

# Yanlış tahminin açıklaması
right_exp = explainer.explain_instance(right, model_reduced.predict, num_features=10)
right_exp.as_pyplot_figure();
plt.title('Tahminin Açıklaması', size = 28);
plt.xlabel('Tahmin üzerindeki etkisi', size = 22);

In [ ]:
right_exp.show_in_notebook(show_predicted_value=False)

Bu durum için doğru değer, gradyan destekli modelimizin doğru bir şekilde elde edildiği 100'dü!

LIME'den gelen görsel yine örnek için her bir özellik değişkeninin önceliğine olan katkısını gösterir. Örneğin, Site EUI'si 62.70'den az olduğu için, bu puanın daha yüksek bir tahminine önemli ölçüde katkıda bulunmuştur. Aynı şekilde, 1927'den önce  inşa edilen yıl da nihai tahminlere olumlu katkıda bulundu.

Bu gibi görselleri gözlemlemek, modelin bir öngörüyü nasıl yaptığı hakkında fikir edinmemize olanak tanır. Bu, büyük olasılıkla modelin kapalı olduğu durumlarda büyük bir değerdir, çünkü hataları daha iyi inceleyebilir ve belki de daha iyi özellikler geliştirebilir veya bir sonraki zaman için tahminleri geliştirmek  adına modelin hiperparametrelerini ayarlayabiliriz. Modelin en fazla kapalı olduğu örnekler, manuel olarak bakmak için ilginç kenar durumları da olabilir. Model, yükseltilmiş Site EUI'sı nedeniyle ilk bina için Enerji Yıldız Puanını büyük ölçüde küçümsedi. Bu nedenle, binanın bu kadar yüksek bir EUI'ye sahip olmasına rağmen neden bu kadar yüksek bir Enerji Yıldız Puanına sahip olduğunu sormak isteyebiliriz. Bir problemin anlaşılmasını sağlamak için makine öğrenimi algoritmasıyla çalışmaya çalıştığımız bu gibi bir süreç, modelin tahminler yapmasına ve tamamen onlara güvenmesine izin vermekten çok daha iyi görünüyor! LIME mükemmel olmasa da, makine öğrenim modellerini açıklamaya yönelik doğru yönde bir adımı temsil eder.

## Tek Karar Ağacının İncelenmesi
Ağaç tabanlı bir topluluk hakkındaki en havalı parçalardan biri, herhangi bir bireysel tahminciye bakabilmemiz. Her ne kadar son modelimiz 800 karar ağacından oluşuyor ve tek bir modele bakmak tüm modelin bir göstergesi değilse de, yine de bir karar ağacının nasıl işlediğine dair genel fikri görmemizi sağlıyor. Oradan, bu ağaçların yüzlerce ağacının, son bir tahminde bulunmak için önceki ağaçların hatalarını ortaya çıkardığını düşünmek doğal bir uzantıdır (bu, gradyanın gerilemesinin nasıl işlediğinin önemli bir basitleştirilmesidir!)

Karar ağaçları ile ilgili detaylı bilgiyi aşağıdaki linklerden ulaşabilirsiniz.

[link-1](https://medium.com/mlreview/gradient-boosting-from-scratch-1e317ae4587d),
[link-2](http://arogozhnikov.github.io/2016/06/24/gradient_boosting_explained.html)

## Sonuç ve Değerlendirme
Makine öğrenme hattının son kısmı en önemlisi olabilir: öğrendiğimiz her şeyi sadece en önemli bulguları vurgulayan kısa bir özet halinde sıkıştırmamız gerekiyor. Şahsen tüm teknik detayların açıklanmasından kaçınmakta zorlanıyorum çünkü tüm işlerin tadını çıkarıyorum. Bununla birlikte, sonuçlarınızı sunduğunuz kişinin tüm ayrıntıları dinlemek için çok fazla zamanı olmayabiliri ve sadece paketlerin tanıtımını yapmanızı istiyor olabilir. Veri bilimi veya makine öğrenimi projesinin en önemli unsurlarını çıkarmayı öğrenmek çok önemli bir beceridir çünkü eğer sonuçlarımız başkaları tarafından anlaşılmadıysa, asla kullanılmayacaktır!

Kendi sonuç kümenizi bulmanızı öneririm, ancak burada 30 saniye içerisinde anlarılmak üzere  en önemli 2 durum belirtild:

 1- Verilen bina enerji verilerini kullanarak, bir makine öğrenme modeli binanın Enerji Yıldızı Puanını 10 puan üzerinden tahmin edebilir.
 2- Enerji Yıldızı Puanını belirlemek için en önemli değişkenler "Enerji Kullanımı Yoğunluğu, Elektrik Kullanımı Yoğunluğu ve Su Kullanımı Yoğunluğu"
 
Eğer herhangi biri detay isterse, tüm uygulama adımlarını kolayca açıklayabilir ve (umarız) iyi belgelenmiş çalışmamızı sunabiliriz. Bir makine öğrenim projesinin bir diğer önemli yanı, tüm kodunuzu yorumlamak ve kodunuz takip etmenizi kolaylaştırmalıdır! İşinize bakabilmeniz ve yaptığınız kararları tamamen anlayabilmeniz için başkasının (ya da birkaç ay içinde kendinizin) olmasını istiyorsunuz. İdeal olarak, tekrar kullanacağı niyetiyle kod yazmalısınız. Projeleri kendimiz yapıyor olsak bile, doğru dokümantasyonu uygulamak iyidir ve bir projeyi tekrar gözden geçirmek istediğinizde hayatınızı kolaylaştıracaktır.

Bu projeyi tamamlamanın zamanı geldi! Bu not defterlerini yazmak ve analiz yapmak için harika bir zaman geçirdim ve umarım bunu okumayı çok seversiniz ve şimdi kendi makine öğrenim projenizi yapmaya başlayabilirisniz. İsterseniz, bu projeyi değiştirerek ve modelleri yenmeye çalışarak başlayabilirsiniz! Unutmayın, tek başına gitmeniz gerektiğini düşünmeyin: aşağıdaki linkerde çok sayıda inanılmaz makine öğrenme kaynağı var ve büyük veri bilimi topluluğunun sayesinde tamamen kendi kendini öğreten birinden geliyor, öğrenmenin en iyi yolu işin içine gir ve işini oraya koymaya başla!

### Makine Öğrenmesi ve Veri Bilimi İçin kaynaklar:

   [link 1](http://shop.oreilly.com/product/0636920052289.do),
   [link 2](https://github.com/ageron/handson-ml),
   [link 3](http://www-bcf.usc.edu/~gareth/ISL/),
   [link 4](https://www.kaggle.com/),
   [link 5](https://www.datacamp.com/),
   [link 6](https://www.dataquest.io/)